In [57]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
import json


# Set up ChromeDriver options
options = Options()
options.headless = True  # Runs Chrome in headless mode.
options.add_argument(r"C:\Users\batuh\AppData\Local\Google\Chrome\User Data")
service = Service(executable_path=r'C:\Users\batuh\Desktop\chromedriver.exe')  # Update with the path to your ChromeDriver




# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)





def get_car_details(url):
    driver.get(url)
    
    car_data = {}

    # Extracting model year from the listing title
    try:
        title_element = driver.find_element(By.CSS_SELECTOR, "h1.listing-title")
        title_text = title_element.text.strip()
        # Assuming the year is always the first four characters
        model_year = title_text.split()[0]
        car_data["Model Year"] = model_year
    except NoSuchElementException:
        car_data["Model Year"] = "Year not found"

    # Extracting details from <dl> elements
    elements = driver.find_elements(By.CSS_SELECTOR, "dl.fancy-description-list dt")
    for element in elements:
        key = element.text.strip()
        value_element = element.find_element(By.XPATH, "./following-sibling::dd[1]")
        value = value_element.text.strip()
        car_data[key] = value

    # Extracting the most recent list price from the price history table
    try:
        price_entries = driver.find_elements(By.CSS_SELECTOR, "div.price-history table tbody tr")
        last_price_element = price_entries[-1].find_element(By.CSS_SELECTOR, "td.list-price")
        last_price = last_price_element.text.strip()
        car_data["Last Listed Price"] = last_price
    except (NoSuchElementException, IndexError):
        car_data["Last Listed Price"] = "Last price not found"

    return car_data




# def scrape_cars(start_url, total_pages=1, batch_size=20):
#     cars = []
#     original_url = start_url  # Store the original start URL
    
#     # Collect all car URLs
#     all_car_urls = []
#     for page in range(total_pages):
#         print(f"Collecting car URLs from page {page + 1}")
#         driver.get(f"{start_url}&page={page + 1}")
#         time.sleep(3)  # Allow some time for the page to load
        
#         # Collect vehicle card URLs
#         vehicle_card_links = driver.find_elements(By.CSS_SELECTOR, "a.vehicle-card-visited-tracking-link")
#         car_urls = [link.get_attribute('href') for link in vehicle_card_links]
#         all_car_urls.extend(car_urls)
    
#     # Process car URLs in batches
#     for i in range(0, len(all_car_urls), batch_size):
#         batch_urls = all_car_urls[i:i+batch_size]
#         print(f"Processing batch {i//batch_size + 1}/{len(all_car_urls)//batch_size}")
#         batch_cars = []
#         for car_url in batch_urls:
#             batch_cars.append(get_car_details(car_url))
#         cars.extend(batch_cars)
    
#     return cars




# URL to start with
start_url = "https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=toyota&maximum_distance=all&mileage_max=&monthly_payment=&page_size=100&sort=best_match_desc&stock_type=used&year_max=&year_min=&zip="

url="https://www.cars.com/vehicledetail/59d0d05b-e54f-449d-8a7c-e52745bdf95d/?attribution_type=isa"

car_list=get_car_details(url)

print(car_list)

# df = pd.DataFrame(car_list)
# df.to_csv('car_data.csv', index=False)
# print("Data saved to 'car_data.csv'.")

# car_details=get_car_details(url)

# Scrape the cars
# car_list = scrape_cars(start_url)

{'Model Year': '2000', 'Exterior color': 'Lunar Mist Metallic', 'Interior color': 'Gray', 'Drivetrain': 'Front-wheel Drive', 'MPG': '23–32', 'Fuel type': 'Gasoline', 'Transmission': 'Automatic', 'Engine': '2.2L I-4 DOHC, regular unleaded, engine with 133HP', 'VIN': '4T1BG22K9YU693763', 'Stock #': '32551', 'Mileage': '134,674 mi.', 'Accidents or damage': 'None reported', '1-owner vehicle': 'No', 'Personal use only': 'Yes', 'Last Listed Price': '$7,000'}


In [58]:
# car_list = scrape_cars(start_url) 

# # Closing the driver
# driver.quit()

# # Creating a DataFrame and saving to CSV
# df = pd.DataFrame(car_list)
# df.to_csv('car_data.csv', index=False)
# print("Data saved to 'car_data.csv'.")

In [59]:
def scrape_cars(start_url):

    cars = []

    # Navigate to the URL
    driver.get(start_url)
    time.sleep(3)  # Allow time for the page to load

    # Collect vehicle card URLs
    vehicle_card_links = driver.find_elements(By.CSS_SELECTOR, "a.vehicle-card-visited-tracking-link")
    all_car_urls = [link.get_attribute('href') for link in vehicle_card_links]

    # Process each car URL to extract details
    for car_url in all_car_urls:
        car_details = get_car_details(car_url)
        cars.append(car_details)
    
    # Close the browser
    driver.quit()

    # Save the data to a JSON file
    with open('car_data2.json', 'w') as f:
        json.dump(cars, f, indent=4)

    return "Data saved to 'car_data.json'."

scrape_cars(start_url)

"Data saved to 'car_data.json'."